In [17]:
#!pip install datasets

from datasets import load_dataset

ds = load_dataset("imdb")
samples = ds["test"]["text"][:100]

with open("eval_small.txt", "a") as out:
    for s in samples:
        out.write(s + "\n")


# Setup

In [20]:
!pip install -r requirements
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/temp/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# RUN MLM

In [26]:
!export CUDA_VISIBLE_DEVICES=1,2,3

#pretrain charbert by bert_base_cased model
#--char_vocab ./data/dict/roberta_char_vocab for robera

!python3 CharBERT/run_lm_finetuning.py \
    --model_type bert \
    --model_name_or_path bert-base-cased \
    --do_train \
    --do_eval \
    --train_data_file CharBERT/data/mlm_datasets/train_small.txt \
    --eval_data_file  CharBERT/data/mlm_datasets/eval_small.txt \
    --term_vocab CharBERT/data/dict/term_vocab \
    --learning_rate 3e-5 \
    --num_train_epochs 10 \
    --char_vocab CharBERT/data/dict/bert_char_vocab \
    --mlm_probability 0.10 \
    --input_nraws 1000 \
    --per_gpu_train_batch_size 4 \
    --per_gpu_eval_batch_size 4 \
    --save_steps 10000 \
    --block_size 384 \
    --mlm \
    --overwrite_output_dir \
    --output_dir model/output/mlm/wiki_20m_eng


2023-12-21 20:06:28.387787: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-21 20:06:28.390581: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 20:06:28.420941: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 20:06:28.420974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 20:06:28.421831: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to